In [1]:
import os, sys

sys.path.insert(0, "/home/j/joppich/.local/lib/python3.11/site-packages/")
print(sys.path)

['/home/j/joppich/.local/lib/python3.11/site-packages/', '/mnt/raidbio/extproj/projekte/regulatory_networks/scripts', '', '/mnt/biosoft/software/python/3.11/lib/python3.11/site-packages', '/mnt/biosoft/software/python/3.11/lib64/python3.11/site-packages', '/usr/lib64/python311.zip', '/usr/lib64/python3.11', '/usr/lib64/python3.11/lib-dynload', '/home/j/joppich/.local/lib/python3.11/site-packages', '/usr/lib64/python3.11/site-packages', '/usr/lib64/python3.11/_import_failed', '/usr/lib/python3.11/site-packages']


In [2]:
import omnipath as op
import pandas as pd

from collections import defaultdict
import networkx as nx

  File "/home/j/joppich/.local/lib/python3.11/site-packages/omnipath/_core/downloader/_downloader.py", line 143, in maybe_download
    res = self._download(req)
          ^^^^^^^^^^^^^^^^^^^
  File "/home/j/joppich/.local/lib/python3.11/site-packages/omnipath/_core/downloader/_downloader.py", line 181, in _download
    resp.raise_for_status()
  File "/mnt/biosoft/software/python/3.11/lib/python3.11/site-packages/requests/models.py", line 1021, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 500 Server Error: Internal Server Error for url: https://omnipathdb.org/queries/enzsub?format=json

  File "/home/j/joppich/.local/lib/python3.11/site-packages/omnipath/_core/downloader/_downloader.py", line 143, in maybe_download
    res = self._download(req)
          ^^^^^^^^^^^^^^^^^^^
  File "/home/j/joppich/.local/lib/python3.11/site-packages/omnipath/_core/downloader/_downloader.py", line 181, in _download
    resp.raise_for_status()
  Fil

In [3]:
opd = op.interactions.AllInteractions().get()

0.00B [00:00, ?B/s]

In [5]:
opd.to_csv("../data/omnipath_nov_2023.tsv", sep="\t")

In [6]:
hgncURL = "https://www.genenames.org/cgi-bin/download/custom?col=gd_hgnc_id&col=gd_app_sym&col=gd_app_name&col=md_prot_id&status=Approved&status=Entry%20Withdrawn&hgnc_dbtag=on&order_by=gd_app_sym_sort&format=text&submit=submit"

hgncTranslation = pd.read_csv(hgncURL, sep="\t")
hgncTranslation.to_csv("../data/omnipath_hgnc_uniprot_nov_2023.tsv", sep="\t")
hgncTranslation

,HGNC ID,Approved symbol,Approved name,UniProt ID(supplied by UniProt)
0,HGNC:5,A1BG,alpha-1-B glycoprotein,P04217
1,HGNC:37133,A1BG-AS1,A1BG antisense RNA 1,NaN
2,HGNC:24086,A1CF,APOBEC1 complementation factor,Q9NQ94
3,HGNC:6,A1S9T,"symbol withdrawn, see [HGNC:12469](/data/gene-...",NaN
4,HGNC:7,A2M,alpha-2-macroglobulin,P01023
...,...,...,...,...
48971,HGNC:25820,ZYG11B,"zyg-11 family member B, cell cycle regulator",Q9C0D3
48972,HGNC:13200,ZYX,zyxin,Q15942
48973,HGNC:51695,ZYXP1,zyxin pseudogene 1,NaN
48974,HGNC:29027,ZZEF1,zinc finger ZZ-type and EF-hand domain contain...,O43149


In [7]:
uniprot2gene = defaultdict(set)

for ri, row in hgncTranslation.iterrows():
    
    hgncGene = row["Approved symbol"]
    uniprotID = row["UniProt ID(supplied by UniProt)"]
    
    if pd.isna(uniprotID):
        continue
    
    uniprot2gene[uniprotID].add(hgncGene)
    

In [8]:
list(opd.type.unique())

['post_translational',
 'transcriptional',
 'post_transcriptional',
 'mirna_transcriptional',
 'lncrna_post_transcriptional',
 'small_molecule_protein']

In [9]:
opd

,source,target,is_directed,is_stimulation,is_inhibition,consensus_direction,consensus_stimulation,consensus_inhibition,curation_effort,references,sources,type,n_sources,n_primary_sources,n_references,references_stripped
0,P0DP23,P48995,True,False,True,True,False,True,3,TRIP:11290752;TRIP:11983166;TRIP:12601176,TRIP,post_translational,1,1,3,11290752;11983166;12601176
1,P0DP25,P48995,True,False,True,True,False,True,3,TRIP:11290752;TRIP:11983166;TRIP:12601176,TRIP,post_translational,1,1,3,11290752;11983166;12601176
2,P0DP24,P48995,True,False,True,True,False,True,3,TRIP:11290752;TRIP:11983166;TRIP:12601176,TRIP,post_translational,1,1,3,11290752;11983166;12601176
3,Q03135,P48995,True,True,False,True,True,False,13,DIP:19897728;HPRD:12732636;IntAct:19897728;Lit...,DIP;HPRD;IntAct;Lit-BM-17;TRIP,post_translational,5,5,8,10980191;12732636;14551243;16822931;18430726;1...
4,P14416,P48995,True,True,False,True,True,False,1,TRIP:18261457,TRIP,post_translational,1,1,1,18261457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282499,5280360,P34995,True,False,False,False,False,False,3,Cellinker:10634944;Cellinker:11999132;Cellinke...,Cellinker;Guide2Pharma_Cellinker,small_molecule_protein,2,1,3,10634944;11999132;16604093
282500,448457,P34995,True,False,False,False,False,False,1,Cellinker:11999132,Cellinker;Guide2Pharma_Cellinker,small_molecule_protein,2,1,1,11999132
282501,5311358,P48544,True,False,False,False,False,False,2,Cellinker:8688080;Cellinker:9804555,Cellinker;Guide2Pharma_Cellinker,small_molecule_protein,2,1,2,8688080;9804555
282502,439260,P35348,True,False,False,False,False,False,3,Cellinker:10433504;Cellinker:7651358;Cellinker...,Cellinker;Guide2Pharma_Cellinker,small_molecule_protein,2,1,3,10433504;7651358;8564227


In [10]:
opd[(opd.type == "transcriptional")]

,source,target,is_directed,is_stimulation,is_inhibition,consensus_direction,consensus_stimulation,consensus_inhibition,curation_effort,references,sources,type,n_sources,n_primary_sources,n_references,references_stripped
134282,P01106,O14746,True,True,False,True,True,False,76,CollecTRI:10022128;CollecTRI:10491298;CollecTR...,CollecTRI;DoRothEA;ExTRI_CollecTRI;HTRI_Collec...,transcriptional,15,4,74,10022128;10491298;10606235;10637317;10723141;1...
134283,P17947,P02818,True,True,False,True,True,False,1,CollecTRI:10022617,CollecTRI;ExTRI_CollecTRI,transcriptional,2,1,1,10022617
134284,COMPLEX:P15407_P17275,P05412,True,True,False,True,True,False,49,CollecTRI:10022869;CollecTRI:10037172;CollecTR...,CollecTRI;ExTRI_CollecTRI;NTNU.Curated_CollecT...,transcriptional,4,1,49,10022869;10037172;10208431;10366004;11281649;1...
134285,COMPLEX:P01100_P05412,P05412,True,True,False,True,True,False,49,CollecTRI:10022869;CollecTRI:10037172;CollecTR...,CollecTRI;ExTRI_CollecTRI;NTNU.Curated_CollecT...,transcriptional,4,1,49,10022869;10037172;10208431;10366004;11281649;1...
134286,COMPLEX:P01100_P17275,P05412,True,True,False,True,True,False,49,CollecTRI:10022869;CollecTRI:10037172;CollecTR...,CollecTRI;ExTRI_CollecTRI;NTNU.Curated_CollecT...,transcriptional,4,1,49,10022869;10037172;10208431;10366004;11281649;1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262495,COMPLEX:Q92731_Q9Y6Q9,P06401,True,True,False,True,True,False,0,NaN,KEGG-MEDICUS,transcriptional,1,1,None,None
262496,COMPLEX:P15336_Q68CJ9,P10145,True,True,False,True,True,False,0,NaN,KEGG-MEDICUS,transcriptional,1,1,None,None
262497,COMPLEX:Q15596_Q92731,P08727,True,True,False,True,True,False,0,NaN,KEGG-MEDICUS,transcriptional,1,1,None,None
262498,COMPLEX:P08047_Q13105,P04629,True,True,False,True,True,False,0,NaN,KEGG-MEDICUS,transcriptional,1,1,None,None


In [ ]:
opd[(opd.is_directed != True)]

In [ ]:
kg = nx.DiGraph()

for ri, row in opd.iterrows():
        
    srcUID = row["source"]
    tgtUID = row["target"]
    
    srcGenes = uniprot2gene.get(srcUID, set())
    tgtGenes = uniprot2gene.get(tgtUID, set())
    
    if len(srcGenes) == 0 or len(tgtGenes) == 0:
        continue
        
    for src in srcGenes:
        kg.add_node(src, type="gene")
        
    for tgt in tgtGenes:
        kg.add_node(tgt, type="gene")

In [ ]:
interactionTypes = {} # stimulation, inhibition
interactionTypes[(False, False)] = "interacts"
interactionTypes[(False, True)] = "represses"
interactionTypes[(True, False)] = "activates"
interactionTypes[(True, True)] = "interacts"

ignoredCount = 0

kg = nx.DiGraph()

for ri, row in opd.iterrows():
        
    srcUID = row["source"]
    tgtUID = row["target"]
    
    srcGenes = uniprot2gene.get(srcUID, set())
    tgtGenes = uniprot2gene.get(tgtUID, set())
    
    if len(srcGenes) == 0 or len(tgtGenes) == 0:
        continue
    
    # consensus_direction -> all resources have same direction
    # consensus_stimulation -> all resources show this as stimulation
    # consensus_inhibition -> all resources show this as consensus_inhibition
    
    # it must have a consensus direction
    if not (row["consensus_direction"]):
        ignoredCount += 1
        continue
    
    # it must either have consensus stimulation or inhibition
    if not (row["consensus_stimulation"] or row["consensus_inhibition"]):
        ignoredCount += 1
        continue

    interactionType = interactionTypes[ (row["consensus_stimulation"], row["consensus_inhibition"]) ]
    
    omnipathEvidences = str(row["references"]).strip().split(";")
    omnipathType = row["type"]
    
    
    for src in srcGenes:
        
        for tgt in tgtGenes:
            
            attrDict = {
                "type": interactionType, "source": "omnipathdb", "omnipath_evidences": omnipathEvidences, "omnipath_type": omnipathType
            }
            
            kg.add_edge( src, tgt, **attrDict )
            
            if not row["is_directed"]:
                kg.add_edge(tgt, src, **attrDict)
    

In [ ]:
print("Ignored Entries", ignoredCount)

In [ ]:
print(kg)